In [1]:
!pip install torch
!pip install transformers

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from tqdm import tqdm
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

device = 'cuda' if torch.cuda.is_available() else 'cpu'

tokenizer_name = "amberoad/bert-multilingual-passage-reranking-msmarco"
model_name = "amberoad/bert-multilingual-passage-reranking-msmarco"

tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device).eval()


def get_bert_scores(queries_to_docnos, docs_dict, queries_dict):
    scores_dict = {}
    for qid in tqdm(queries_to_docnos, desc="BERT", total=len(queries_to_docnos)):
        for docno in queries_to_docnos[qid]:
            query_text = queries_dict[qid]
            doc_text = docs_dict[docno]
            ret = tokenizer.encode_plus(query_text,
                                        doc_text,
                                        max_length=512,
                                        truncation=True,
                                        return_token_type_ids=True,
                                        return_tensors='pt')

            with torch.cuda.amp.autocast(enabled=False):
                input_ids = ret['input_ids'].to(device)
                tt_ids = ret['token_type_ids'].to(device)
                output, = model(input_ids, token_type_ids=tt_ids, return_dict=False)
                if output.size(1) > 1:
                    score = torch.nn.functional.log_softmax(output, 1)[0, -1].item()
                else:
                    score = output.item()

            scores_dict[docno] = score

    return scores_dict


C:\Users\nivba\anaconda3\envs\thesis_env_39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
queries_dict = {102: "Eurovision Song Contest", 154: "iPhone"}
docs_dict = {"doc1": "The Eurovision Song Contest, often known simply as Eurovision or by its initialism ESC, is an international song competition organised annually by the European Broadcasting Union", 
             "doc2": "London, the capital of England and the United Kingdom, is a 21st-century city with history stretching back to Roman times. At its centre stand the imposing Houses of Parliament, the iconic ‘Big Ben’ clock tower and Westminster Abbey, site of British monarch coronations. Across the Thames River, the London Eye observation wheel provides panoramic views of the South Bank cultural complex, and the entire city.", 
             "doc3": "we like pizza.",
             "doc4": "An iPhone is a line of smartphones developed and sold by Apple Inc., offering a range of advanced features, sleek design, and seamless integration with Apple's ecosystem, including access to the App Store, iCloud services, and exclusive software like Siri, making it a popular choice among smartphone users worldwide."}

queries_to_docnos = {102: ["doc1", "doc3"], 154: ["doc2", "doc4"]}

scores = get_bert_scores(queries_to_docnos, docs_dict, queries_dict)          

BERT: 100%|█████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  5.92it/s]


In [8]:
for query in queries_dict:
    print("Query: ", queries_dict[query])
    docs = sorted(queries_to_docnos[query], key=lambda x: -scores[x])
    for i, docno in enumerate(docs):
        print(f"{i+1}.\t{docs_dict[docno]}, {scores[docno]}")
    print()

Query:  Eurovision Song Contest
1.	The Eurovision Song Contest, often known simply as Eurovision or by its initialism ESC, is an international song competition organised annually by the European Broadcasting Union, -0.0014832934830337763
2.	we like pizza., -10.465782165527344

Query:  iPhone
1.	An iPhone is a line of smartphones developed and sold by Apple Inc., offering a range of advanced features, sleek design, and seamless integration with Apple's ecosystem, including access to the App Store, iCloud services, and exclusive software like Siri, making it a popular choice among smartphone users worldwide., -0.0026964517310261726
2.	London, the capital of England and the United Kingdom, is a 21st-century city with history stretching back to Roman times. At its centre stand the imposing Houses of Parliament, the iconic ‘Big Ben’ clock tower and Westminster Abbey, site of British monarch coronations. Across the Thames River, the London Eye observation wheel provides panoramic views of th